# Pre processing for skin lesion dataset

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

# Visualização auxiliar
def show_images(img_list, titles=None, cmap='gray'):
    plt.figure(figsize=(15, 5))
    for i, img in enumerate(img_list):
        plt.subplot(1, len(img_list), i+1)
        plt.imshow(img, cmap=cmap)
        if titles:
            plt.title(titles[i])
        plt.axis('off')
    plt.show()

# Etapa 1: Redimensionamento
class Resize(BaseEstimator, TransformerMixin):
    def __init__(self, size=(128, 128)):
        self.size = size
        
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [cv2.resize(img, self.size) for img in X]

# Etapa 2: Filtro Gaussiano (remoção de ruído)
class GaussianBlur(BaseEstimator, TransformerMixin):
    def __init__(self, ksize=(5, 5), sigma=0):
        self.ksize = ksize
        self.sigma = sigma

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [cv2.GaussianBlur(img, self.ksize, self.sigma) for img in X]

# Etapa 3: CLAHE (Equalização de histograma adaptativa)
class CLAHE(BaseEstimator, TransformerMixin):
    def __init__(self, clip_limit=2.0, tile_grid_size=(8, 8)):
        self.clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [self.clahe.apply(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)) for img in X]

# Etapa 4: Segmentação simples usando Otsu
class OtsuThreshold(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        binarized = []
        for img in X:
            _, binary = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            binarized.append(binary)
        return binarized

# Leitura da imagem de exemplo
img_path = '/mnt/data/ISIC_0052068.jpg'  # ou o caminho local da sua imagem
original = cv2.imread(img_path)

# Criando pipeline
pipeline = Pipeline([
    ('resize', Resize((128, 128))),
    ('blur', GaussianBlur((5, 5))),
    ('clahe', CLAHE()),
    ('threshold', OtsuThreshold())
])

# Aplicar pipeline
processed_images = pipeline.fit_transform([original])

# Visualização
show_images(
    [original, *processed_images],
    titles=["Original", "Redimensionada + Desfocada + CLAHE + Otsu"]
)
